# BepiColombo SPICE Kernel Dataset Status
SKD version: {skd_version} <br>
Creation: {current_time} by ESA SPICE Service (ESAC/ESA). <br>
Bepi Test specification: SPICE-CROSS-TN-190322 <br>


## Startup
Prepare the testing environment at first stage, so if any validation step fails stopping the execution, the tests are already defined and in fail state.

In [1]:
from spival.classes.history import TestHistory

test_history = TestHistory()
test_history.add_test('XM-C1', 'Metakernel is valid', 'Consistency')
test_history.add_test('XM-C2', 'Frame chain', 'Consistency')
test_history.add_test('XM-C3', 'Rotation matrices', 'Consistency')
test_history.add_test('XM-C4', 'Fields of view', 'Consistency')
test_history.add_test('XM-V1', 'Time deviation', 'Validity/Error', threshold='500ms')
test_history.add_test('XM-V1-BIS', 'Time correlation', 'Validity/Error', threshold='500ms')
test_history.add_test('XM-Q2', 'S/C Clock Drift', 'Quality/Evolution')
test_history.add_test('XM-V3', 'Predicted Quaternions to CK error', 'Validity/Warning', threshold='50 mdeg')
test_history.add_test('XM-V4', 'Measured Quaternions to CK error', 'Validity/Warning', threshold='5 mdeg')
test_history.add_test('XM-V5', 'OEM to SPK error', 'Validity/Error', threshold='~100m')
test_history.add_test('XM-V7-MPO', 'MPO Solar Array (SA) Angles', 'Validity/Error', threshold='100 mdeg')
test_history.add_test('XM-V7-MTM', 'MTM Solar Array (SA) Angles', 'Validity/Error', threshold='100 mdeg')
test_history.add_test('XM-Q3', 'Attitude Error', 'Quality/Data')
test_history.add_test('XM-Q3-BIS', 'SCC vs FCP Attitude Error', 'Quality/Data')
test_history.add_test('XM-Q4', 'MPO Orientation (quaternions w.r.. J2000)', 'Quality/Data')
test_history.add_test('XM-Q5-MPO', 'MPO Solar Array (SA) Angles', 'Quality/Data')
test_history.add_test('XM-Q5-MTM', 'MTM Solar Array (SA) Angles', 'Quality/Data')
test_history.add_test('XM-Q1-MPO', 'MPO SA Solar Aspect Angle', 'Quality/Data')
test_history.add_test('XM-Q1-MTM', 'MTM SA Solar Aspect Angle', 'Quality/Data')
test_history.add_test('XM-Q6', 'MPO High Gain Antenna Angles', 'Quality/Data')
test_history.add_test('XM-Q7', 'MPO High Gain Antenna - Earth Angle', 'Quality/Data')
test_history.add_test('BC-Q2', 'MPO Medium Gain Antenna Angles', 'Quality/Data')
test_history.add_test('BC-Q8', 'MPO Medium Gain Antenna - Earth Angle', 'Quality/Data')
#test_history.add_test('BC-V1', 'MPO Phebus Misc Scanner measured position vs CK error', 'Validity/Warning', threshold='100 mdeg')
test_history.add_test('BC-V3-MERCURY_1', 'MERCURY Flyby #1 01-OCT-2021 — CA altitudes', 'Quality/Evolution')
test_history.add_test('BC-V3-MERCURY_2', 'MERCURY Flyby #2 23-JUN-2022 — CA altitudes', 'Quality/Evolution')
test_history.add_test('BC-V3-MERCURY_3', 'MERCURY Flyby #3 19-JUN-2023 — CA altitudes', 'Quality/Evolution')
test_history.add_test('XM-V10', 'Diff of latest orbit w.r.t. previous orbit', 'Quality/Evolution')

### XM-C1 - Test Metakernel is valid 
The metakernel is loaded, the scenario covers a week with a finish time set by the end of coverage of the latest Measured Attitude Kernel.

In [ ]:
from spiops import spiops
import spiceypy

spiops.load('{metakernel}')               
mission_config = spiops.load_config('{config_file}')               

test_history.set_test_result('XM-C1', True)

start_time = '{start_time_measured}'                                # Start time
finish_time ='{finish_time_measured}'                               # End time

interval = spiops.TimeWindow(start_time, finish_time,resolution=60) # spiops object TimeWindow generated
sun = spiops.Target('SUN', time=interval, frame='IAU_SUN')          # spiops object Target Sun generated
mpo = spiops.Observer('MPO', time=interval, target=sun, mission_config=mission_config)  # spiops object Observer MPO generated
mtm = spiops.Observer('MTM', time=interval, target=sun, mission_config=mission_config)  # spiops object Observer MTM generated
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### <center><span style='color:Green'> Metakernel loaded successfully.  </span></center>

### XM-C2 - Frame chain
Ensure that frame changes operations can be performed at any time (incl. instrument frames), meaning all frames are connected.

In [ ]:
plan_mk = '{metakernel}'.replace('ops', 'plan')
spiops.load(plan_mk)
frm_start_time = '{start_time_measured}'                                # Start time
frm_finish_time ='{finish_time_measured}'                               # End time
num_samples = 100

all_frames_ok = spiops.check_frame_chain(frm_start_time, frm_finish_time, num_samples, ignore_frames=['@SW@MMO_TILT', '@SW@MMO_SPIN'])
test_history.set_test_result('XM-C2', all_frames_ok)

### XM-C3 - Rotation matrices
Check that all TK frames defined with matrices are defined with proper rotation matrices.

In [ ]:
all_matrices_ok = spiops.check_rotation_matrices()
test_history.set_test_result('XM-C3', all_matrices_ok)


### XM-C4 - Fields of view
Check that all instruments fields of view are defined properly.

In [ ]:
all_fovs_ok = spiops.check_fovs(max_angle_deg=89.99994)
test_history.set_test_result('XM-C4', all_fovs_ok)
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V1 - Time deviation
Downloads the time deviation files for a given time range and computes the time difference between the UTC timestamp of packet calculated with SPICE (1st column) and the UTC timestamp of packet calculated from SCOS2K header (3rd column)

In [ ]:
spiops.load('{metakernel}')
max_time_diff = spiops.time_deviation('MPO', start_time, finish_time, plot_style='line', notebook=True)
test_history.set_test_result('XM-V1', (max_time_diff != None) and (max_time_diff < 500)) # 500 milliseconds
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V1-BIS - Time correlation
Downloads a telemetry file of a given CK and computes the time difference between the UTC time (1st column) and the clock string (2nd column) in milliseconds

In [ ]:
spiops.load('{metakernel}')
target_ck = '{measured_ck}'
max_time_diff = spiops.time_correlation('MPO', target_ck, plot_style='line', notebook=True)
test_history.set_test_result('XM-V1-BIS', (max_time_diff != None) and (max_time_diff < 500)) # 500 milliseconds
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-Q2 - S/C Clock Drift 

The following plot shows the drift of the S/C Clock.

In [ ]:
spiops.load('{metakernel}')
mpo.Plot('clock_drift', notebook=True)
test_history.set_test_result('XM-Q2', True)
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

## Coverage

The coverage provided by the SPK and CK files is displayed for the Operational and the Planning meta-kernels. 

The Data type designation includes a reference to the originator of the data, the type of data and the reference period is provided. This is a three letter acronym. The first letter defines the file originator:
*    s: Science Operation
*    m: Mission Analysis
*    f: Flight Dynamic

the second letter defines the reference period of the data:
*    c: Cruise phase reference
*    l: Science phase Long term reference
*    m: Science phase Medium term reference
*    s: Science phase Short term reference
*    o: Undefined reference

the third letter indicates the type of data
*    <span style='color:Orange'> p: Predicted data</span>
*    <span style='color:Green'> r: Reconstructed data</span>
*    <span style='color:Red'> t: Test data</span>
*    <span style='color:Purple'> c: Commanded data (from Housekeeping Telemetry)</span>
*    <span style='color:Blue'> m: Measured data (from Housekeeping Telemetry)</span>


In [ ]:
spiops.ck_coverage_timeline('{metakernel}', ['MPO_SPACECRAFT', 'MMO_SPACECRAFT', 'MTM_SPACECRAFT'])
spiops.ck_coverage_timeline('{metakernel}'.replace('ops', 'plan'), ['MPO_SPACECRAFT', 'MMO_SPACECRAFT', 'MTM_SPACECRAFT'])
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

In [ ]:
spiops.spk_coverage_timeline('{metakernel}', ['MPO', 'MMO', 'MTM'])
spiops.spk_coverage_timeline('{metakernel}'.replace('ops', 'plan'), ['MPO', 'MMO', 'MTM'])
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V3 - Predicted Quaternions to CK error
Comparison of spacecraft orientation (quaternion) between source AEM Quaternions and generated CK

In [ ]:
spiops.load('{metakernel}'.replace('ops', 'plan'))
target_ck = '{predicted_ck}'
max_err = spiops.ckVsAEM('MPO', target_ck, plot_style='line', notebook=True)
test_history.set_test_result('XM-V3', (max_err != None) and (max_err < 50)) # 50mdeg
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V4 - Measured Quaternions to CK error
Comparison of spacecraft orientation (quaternion) between source AOCS Measured Quaternions and generated CK 

In [ ]:
spiops.load('{metakernel}')
target_ck = '{measured_ck}'
max_err = spiops.ckVsAocs('MPO', target_ck, plot_style='line', notebook=True)
test_history.set_test_result('XM-V4', (max_err != None) and (max_err < 5)) # 5mdeg
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V5 - OEM to SPK error
Comparison of spacecraft position and velocity between source OEM and generated SPK 

In [ ]:
spiops.load('{metakernel}')
target_spk = '{reconstructed_spk}'
max_pos_err, max_vel_err, discontinuities = spiops.spkVsOem('MPO', target_spk, plot_style='line', notebook=True)
test_history.set_test_result('XM-V5', (max_pos_err != None) and (max_pos_err < 0.1) and (len(discontinuities) == 0))
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V7-MPO - MPO Solar Array (SA) Angles
Comparison of MPO SADM Angle from HK TM and SPICE CK

In [ ]:
spiops.load('{metakernel}')
max_err = spiops.saa_vs_hk_sa_position('MPO', plot_style='line', notebook=True)
test_history.set_test_result('XM-V7-MPO', (max_err != None) and (max_err < 100)) # 100mdeg
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-V7-MTM - MTM Solar Array (SA) Angles
Comparison of MTM SA Angle from HK TM and SPICE CK

In [ ]:
spiops.load('{metakernel}')
max_err = spiops.saa_vs_hk_sa_position('MTM', plot_style='line', notebook=True)
test_history.set_test_result('XM-V7-MTM', (max_err != None) and (max_err < 100)) # 100mdeg
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-Q3 - Attitude Error
Comparison of +Z axis orientation between predicted and measured attitude in arcseconds 

In [ ]:
spiops.load('{metakernel}'.replace('ops', 'plan'))
measured_ck = '{measured_ck}'
predicted_ck = '{predicted_ck}'
resolution = 4

res = spiops.ckdiff_error(measured_ck, predicted_ck, 'MPO_SPACECRAFT', 'J2000', resolution, 0.001, 
                    plot_style='circle', utc_start=start_time, utc_finish=finish_time, notebook=True)
test_history.set_test_result('XM-Q3', res is not None)
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

### XM-Q3-BIS - SCC vs FCP Attitude Error
Comparison of +Z axis orientation between SOC commanded and FDy predicted attitude in arcseconds 

In [ ]:
spiops.load('{metakernel}')
commanded_ck = '{commanded_ck}'
predicted_ck = '{predicted_ck}'
resolution = 4

res = spiops.ckdiff_error(commanded_ck, predicted_ck, 'MPO_SPACECRAFT', 'J2000', resolution, 0.001, 
                    plot_style='circle', utc_start=start_time, utc_finish=finish_time, notebook=True)
test_history.set_test_result('XM-Q3-BIS', res is not None)
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

## S/C Structures Orientation
Quaternions, MTM and MPO Solar Array Orientation and Solar Aspect Angle, High Gain Antenna Orientation and HGA boresight-Earth Angle.

### XM-Q4 - MPO Orientation (quaternions w.r.. J2000)

In [ ]:
spiops.load('{metakernel}')
mpo.Plot('quaternions', notebook=True)
test_history.set_test_result('XM-Q4', True)

### XM-Q5-MPO - MPO Solar Array (SA) Angles 

In [ ]:
mpo.Plot('sa_ang', notebook=True)
test_history.set_test_result('XM-Q5-MPO', True)

### XM-Q5-MTM - MTM Solar Array (SA) Angles

In [ ]:
mtm.Plot('sa_ang', notebook=True)
test_history.set_test_result('XM-Q5-MTM', True)

### XM-Q1-MPO - MPO SA Solar Aspect Angle

In [ ]:
mpo.Plot('saa_sa', notebook=True)
test_history.set_test_result('XM-Q1-MPO', True)

### XM-Q1-MTM - MTM SA Solar Aspect Angle

In [ ]:
mtm.Plot('saa_sa', notebook=True)
test_history.set_test_result('XM-Q1-MTM', True)

### XM-Q6 - MPO High Gain Antenna Angles

In [ ]:
mpo.Plot('hga_angles', notebook=True)
test_history.set_test_result('XM-Q6', True)       

### XM-Q7- MPO High Gain Antenna - Earth Angle

In [ ]:
mpo.Plot('hga_earth', notebook=True)
test_history.set_test_result('XM-Q7', True)

### BC-Q2 - MPO Medium Gain Antenna Angles

In [ ]:
mpo.Plot('mga_angles', notebook=True)
test_history.set_test_result('BC-Q2', True)

### BC-Q8- MPO Medium Gain Antenna - Earth Angle

In [ ]:
mpo.Plot('mga_earth', notebook=True)
test_history.set_test_result('BC-Q8', True)

### BC-V1 - MPO Phebus Misc Scanner measured position vs CK error

In [ ]:
#max_err = spiops.phe_misc_scanner_ck_vs_hk_position(plot_style='line', notebook=True)
#test_history.set_test_result('BC-V1', (max_err != None) and (max_err < 100)) # 100mdeg

### BC-V3-MERCURY_1 - MERCURY Flyby #1 01-OCT-2021 — CA altitudes

In [ ]:
target = 'MERCURY'
spk_expression = 'bc_mpo_fcp_?????_????????_????????_v??.bsp'
num_spk_files = 10
from_date = '2021-10-01T20:00:00' 
to_date = '2021-10-02T02:00:00'
distance_flyby = 30000
num_samples = 5000
max_flyby_alt = spiops.flyby_ca_altitudes('MPO', target, spk_expression, num_spk_files, 
                                           from_date, to_date, distance_flyby, num_samples,
                                           plot_style='line', notebook=True, plot_prefix="1st")
test_history.set_test_result('BC-V3-MERCURY_1', (max_flyby_alt != None))

### BC-V3-MERCURY_2 - MERCURY Flyby #2 23-JUN-2022 — CA altitudes

In [ ]:
target = 'MERCURY'
spk_expression = 'bc_mpo_fcp_?????_????????_????????_v??.bsp'
num_spk_files = 10
from_date = '2022-06-23T05:30:00' 
to_date = '2022-06-23T13:35:00'
distance_flyby = 30000
num_samples = 5000
max_flyby_alt = spiops.flyby_ca_altitudes('MPO', target, spk_expression, num_spk_files, 
                                           from_date, to_date, distance_flyby, num_samples,
                                           plot_style='line', notebook=True, plot_prefix="2nd")
test_history.set_test_result('BC-V3-MERCURY_2', (max_flyby_alt != None))

### BC-V3-MERCURY_3 - MERCURY Flyby #3 19-JUN-2023 — CA altitudes

In [ ]:
target = 'MERCURY'
spk_expression = 'bc_mpo_fcp_?????_????????_????????_v??.bsp'
num_spk_files = 10
from_date = '2023-06-19T17:00:00' 
to_date = '2023-06-20T01:00:00'
distance_flyby = 30000
num_samples = 5000
max_flyby_alt = spiops.flyby_ca_altitudes('MPO', target, spk_expression, num_spk_files, 
                                           from_date, to_date, distance_flyby, num_samples,
                                           plot_style='line', notebook=True, plot_prefix="3rd")
test_history.set_test_result('BC-V3-MERCURY_3', (max_flyby_alt != None))

### XM-V10 - Diff of latest orbit w.r.t. previous orbit

In [ ]:
spk_expression = 'bc_mpo_fcp_?????_????????_????????_v??.bsp'
num_samples = 5000
max_position_error = spiops.spk_diff('MPO', spk_expression, num_samples, notebook=True)
test_history.set_test_result('XM-V10', (max_position_error != None))

In [ ]:
# Unload ops MK
spiceypy.kclear()  # Avoid any plan kernel in the kernel pool

In [ ]:
# Show validation results
test_history.show_tests()